# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [33]:
!pip install langchain

In [34]:
!pip install langchain_huggingface langchain-community langchain_chroma Chroma langchain_groq

In [35]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  dataset.zip
replace Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 
error:  invalid response [{ENTER}]
replace Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: Dataset.csv             


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [36]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain

ERROR: Could not find a version that satisfies the requirement LLMChain (from versions: none)
ERROR: No matching distribution found for LLMChain


# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [37]:
import os
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
import markdown
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_groq import ChatGroq

In [38]:
df= pd.read_csv('/content/Dataset.csv')
df

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال
...,...,...
99,القيام بأعمال الطرق قبل التنسيق مع الإدارة الم...,الغرامة المالية 5000 - 10000 ريال
100,سماح أصحاب الحيوانات بعبور حيواناتهم من غير ال...,الغرامة المالية 5000 - 10000 ريال
101,استخدام لوحات غير عائدة للمركبة.,الغرامة المالية 5000 - 10000 ريال
102,طمس رقم هيكل المركبة أو محاولة طمسه.,الغرامة المالية 5000 - 10000 ريال


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [39]:
directory = 'data/markdown_files'
os.makedirs(directory, exist_ok=True)

In [40]:
for i in range(0, 104):

    title = df['الغرامة'].iloc[i]
    content = df['المخالفة'].iloc[i]

    markdown_content = f"# {title}\n\n"
    markdown_content += f"{content}\n\n"

    with open(f'{directory}/{i}.md', 'w', encoding='utf-8') as file:
        file.write(markdown_content)

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [41]:
markdown_texts = []
for filename in os.listdir(directory):
  if filename.endswith(".md"):
    with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
      markdown_content = file.read()
      html_content = markdown.markdown(markdown_content)
      markdown_texts.append(html_content)

In [42]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=50)
documents = text_splitter.create_documents(markdown_texts)

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [43]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma.from_documents(documents, embedding_function, persist_directory="./chroma_db")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [44]:
rag_prompt_template = """
السياق: {context}
السؤال: {query}
الإجابة:
يرجى تقديم اجابة تفصيلية باللغة العربيه فقط
"""

## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [45]:
groq_api_key = "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"
llm = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama3-8b-8192")

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [46]:
prompt_template = PromptTemplate(
    template=rag_prompt_template, input_variables=["context", "query"]
)
llm_chain = LLMChain(llm=llm,
                 prompt=prompt_template,
                 )

In [47]:
prompt = PromptTemplate(template=rag_prompt_template, input_variables=["context", "query"])
llm_chain = LLMChain(llm=llm, prompt=prompt)

## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [48]:
def query_rag(user_query):
    relevant_context = db.similarity_search(user_query)
    context = "\n".join([doc.page_content for doc in relevant_context])
    response = llm_chain.run(context=context, query=user_query)

    return response


## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [49]:
def run_inference():
    response = query_rag("ماهي الغرامة على القيادة بدون رخصة في السعودية؟")
    print("إجابة: ", response)
run_inference()

إجابة:  في المملكة العربية السعودية، الغرامة على القيادة بدون رخصة أو القيادة دون الامتثال لشروط الرخصة هي 500 ريال سعودي، بالإضافة إلى إيقاف السيارة لمدة 30 يوماً. كما أن هناك أيضاً إمكانية الإقامة في السجن لمدة تصل إلى 15 يوماً في حالة المخالفة المتكررة.

كما أن هناك أيضاً الغرامة على القيادة بدون رخصة لغير السعوديين، والتي تصل إلى 1000 ريال سعودي، بالإضافة إلى إيقاف السيارة لمدة 30 يوماً.

يجب على كل سائق أن يمتثل لشروط الرخصة والقوانين المرتبطة بها، ويتأكد من أن رخصته سارية المفعول والمتوافقة مع شروط السير في المملكة العربية السعودية.


In [52]:
def run_inference():
    response = query_rag("كم اقصى سرعة مسموحة  في المسار الايسر؟")
    print("إجابة: ", response)
run_inference()

إجابة:  من خلال دراسة السؤال والسياق، نجد أن السؤال يطلب معرفة أقصى سرعة مسموحة في المسار الايسر. لذلك، من المهم أن نستعرض القوانين والتعليمات المرتبطة بالقيادة على الطرق والمسارات.

في المملكة العربية السعودية، يوجد قانون رقم 34 لسنة 1990 بشأن المرور والسلامة على الطرق، والذي يحدد القواعد والتعليمات الخاصة بالقيادة على الطرق. وفقاً لهذا القانون، يمنع القيادة على الأرصفة أو المسارات التي يمنع القيادة، ما عدا في الحالات الم specifi ed في القانون.

في حالة التقاطع، يجب أن يمر المركبة القادمة من اليمين أولاً عند الوصول إلى التقاطع، كما هو مذكور في السؤال. في هذه الحالة، يجب أن يتبع المركبة الأخرى التي تاتي من اليسار قواعد القيادة المعمول بها في التقاطع.

من حيث سرعة القيادة، يوجد حد أقصى مسموح به في المسار الايسر، والذي يتراوح بين 50-80 كم/ساعة، حسب نوع المسار والظروف الجوية والرطوبة. ومع ذلك، يجب أن يتبع السائق قواعد القيادة المعمول بها في التقاطع، مثل الوقوف قبل التقاطع، والتحكم في السرعة، والمرور بأمان.

في النهاية، أقصى سرعة مسموحة في المسار الايسر هي 60 كم/ساعة، وفقاً لما هو مذكور في

In [54]:
def run_inference():
    response = query_rag("ماهي  مخالفة قطع الاشارة؟")
    print("إجابة: ", response)
run_inference()

إجابة:  مخالفة قطع الاشارة هي إهمال أو تجاهل الاشارة التي تُقدم من قبل سلطة النقل أو من قبل شخص آخر مسؤول عن إدارة حركة المرور، والتي تُقدم لتحذير أو توجيه المركبات أو لمنعها من عبور أو مرور من منطقة معينة.

في السياق المذكور، مخالفة قطع الاشارة يمكن أن تكون قيادة المركبة داخل الأنفاق من غير إضاءة، مما يهدد سلامة السائقين والمركبات الأخرى التي تنتظر أو تنتقل داخل الأنفاق. هذا النوع من المخالفات يُعد من أخطر المخالفات التي تُقدم على الطرق، لأنها تُقدم خطراً مباشراً على حياة السائقين والمركبات الأخرى.
